<a href="https://colab.research.google.com/github/jubeam/my_js_project/blob/master/planesnet_class_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#0. 사용할 패키지 불러오기
import numpy as np
import glob
import os.path as path
from scipy import misc
from google.colab import drive

#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Flatten
#from keras.layers.convolutional import Conv2D
#from keras.layers.convolutional import MaxPooling2D
#from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
#np.random.seed(3)

# 1. 데이터 생성하기


drive.mount('/gdrive', force_remount=True)
IMAGE_PATH= '/gdrive/My Drive/Colab Notebooks/planesnet'
file_paths= glob.glob(path.join(IMAGE_PATH, '*.png'))

#Load the images
images = [misc.imread(path)for ptah in file_paths]
images = np.asarray(images)

#Get the image size

image_size=np.asarray([images.shape[1], images.shape[2],images.shape[3])])



train_generator = train_datagen.flow_from_directory(
        '/gdrive/My Drive/Colab Notebooks/WDF/train',
        target_size=(21, 21),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/gdrive/My Drive/Colab Notebooks/WDF/test',
        target_size=(21, 21),    
        batch_size=3,
       class_mode='categorical')
real_test_datagen = ImageDataGenerator(rescale=1./255)
real_test_generator = real_test_datagen.flow_from_directory(
        '/gdrive/My Drive/Colab Notebooks/WDF/real_test',
        target_size=(21, 21),    
        batch_size=3,
       class_mode='categorical')

# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(21,21,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의
#hist = model.fit(X_train, Y_train, epochs=3000, batch_size=10, validation_data=(X_val, Y_val), callbacks=[early_stopping])
hist= model.fit_generator(
        train_generator,
        steps_per_epoch=15, # * 100,
        epochs=10, # 100,
        validation_data=test_generator,
        validation_steps=5,
        callbacks=[early_stopping]    
        )
# 5. 모델 학습 과정 표시하기
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()
# 6. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 7. 모델 사용하기
print("-- Predict --")

#X=[]
#filenames=[]
#imge_w=7
#imge_h=7
#pixels=imge_h * imge_w *3
#file_dir='/gdrive/My Drive/Colab Notebooks/WDF/real_test'
#files = glob.glob(file_dir+"/*/*.*")
#for i, f in enumerate(files):
#  img= Image.open(f)
#  img= img.convert("RGB")
#  img=img.resize((imge_w,imge_h))
#  data=np.asarray(img)
#  filenames.append(f)
#  X.append(data)

#prediction=model.predict(X)

output = model.predict_generator(real_test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(real_test_generator.class_indices)
print(output)